# Hello LangChain

In [3]:
from langchain_openai import ChatOpenAI

In [6]:
llm = ChatOpenAI()

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [3]:
from langchain_core.prompts import ChatPromptTemplate

In [4]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [5]:
chain = prompt | llm 

In [6]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content="Langsmith can be a valuable resource for testing in various ways. Here are a few examples:\n\n1. Test Case Documentation: Langsmith can assist in writing clear and comprehensive test case documentation. This includes defining test scenarios, steps, expected results, and any necessary data or preconditions. Well-documented test cases help ensure that all aspects of the software are thoroughly tested.\n\n2. Test Plan Creation: Langsmith can help create test plans that outline the overall testing strategy, including objectives, scope, resources, and timelines. A well-defined test plan ensures that testing efforts are organized and focused, leading to more effective testing.\n\n3. User Acceptance Testing (UAT): Langsmith can assist in writing UAT scripts and test cases. UAT ensures that the software meets the requirements and expectations of end-users. By providing clear instructions and expected results, Langsmith can help ensure a smooth UAT process.\n\n4. Automated Te

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
chain = prompt | llm | output_parser

In [9]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can be a valuable tool for testing in several ways:\n\n1. Test Case Generation: Langsmith can automatically generate test cases based on the specifications provided. It uses natural language processing to understand the requirements and generate test cases that cover all possible scenarios. This saves time and effort in manually creating test cases, ensuring better coverage and reducing the risk of missing any critical test scenarios.\n\n2. Test Data Generation: Testing often requires a variety of test data to cover different scenarios. Langsmith can generate realistic and diverse test data based on the specifications. It can generate data for various data types, including strings, numbers, dates, and more. This helps in creating a comprehensive test suite with a wide range of data inputs.\n\n3. Test Documentation: Langsmith can automatically generate test documentation by extracting relevant information from the specifications. It can create test plans, test scripts, and te

In [10]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [12]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [13]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing users to visualize test results.'

In [16]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [17]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by providing the ability to run chains over datasets and visualize the outputs. It simplifies dataset uploading and allows users to easily review the results of running chains over data points. Users can assign feedback to runs and mark them as correct or incorrect directly in the web app. LangSmith also offers evaluators that can be specified during test runs to evaluate the results. Additionally, LangSmith provides annotation queues for manual review and annotation of runs, allowing users to assess subjective qualities and validate auto-evaluated runs.
